### LRU Cache

Least Recently Used Cache

Python Implementation: https://github.com/python/cpython/blob/3.7/Lib/functools.py

In [1]:
from collections import deque, defaultdict

class LRUCache:
    def __init__(self, capacity):
        """
        :type capacity: int
        """
        self._capacity = capacity
        self._cache = {}
        self._counter= defaultdict(int)
        
        self._access_log = deque()
        
    def _clean_up_end_of_access_log(self):
        while self._access_log:
            last_key = self._access_log[-1]
            if last_key not in self._cache:
                # remove deleted items from access_log
                print(' - removed deleted item from end of access log: ', last_key)
                self._access_log.pop()
            elif self._counter[last_key] != 1:
                # clean up dups at the end of log
                self._access_log.pop()
                self._counter[last_key] -= 1
                print(' - removed dup item from end of access log: ', last_key)
            else:
                break
        
    def get(self, key):
        """
        :type key: int
        :rtype: int
        """
        val = self._cache.get(key, -1)
        
        if val != -1:
            self._access_log.appendleft(key)
            self._counter[key] += 1
            self._clean_up_end_of_access_log()
            
                    
        print('get(%s) -> %s \t | Cache(%s): %s, Log: %s, Counter: %s' % (key, val, len(self._cache), self._cache, self._access_log, self._counter))
        return val

    def put(self, key, value):
        """
        :type key: int
        :type value: int
        :rtype: void
        """
        self._cache[key] = value
        self._access_log.appendleft(key)
        self._counter[key] += 1
            
        self._clean_up_end_of_access_log()
        
        if self._counter[key] == 1 and len(self._cache) > self._capacity:
            # Remove an item 
            key_to_remove = self._access_log.pop()
            assert self._counter[key_to_remove] == 1
            del self._counter[key_to_remove]
            del self._cache[key_to_remove]
            print(' - removed item from cache: ', key_to_remove)        
        
        print("put(%s, %s) \t | Cache(%s): %s, Log: %s, Counter: %s" % (key, value, len(self._cache), self._cache, self._access_log, self._counter))
        
        
        
lru_cache = LRUCache(2)
lru_cache.put(1, 1)
lru_cache.put(2, 2)
assert lru_cache.get(1) == 1
lru_cache.put(3, 3)
assert lru_cache.get(2) == -1
lru_cache.put(4, 4)
assert lru_cache.get(1) == -1
assert lru_cache.get(3) == 3
assert lru_cache.get(4) == 4


put(1, 1) 	 | Cache(1): {1: 1}, Log: deque([1]), Counter: defaultdict(<class 'int'>, {1: 1})
put(2, 2) 	 | Cache(2): {1: 1, 2: 2}, Log: deque([2, 1]), Counter: defaultdict(<class 'int'>, {1: 1, 2: 1})
 - removed dup item from end of access log:  1
get(1) -> 1 	 | Cache(2): {1: 1, 2: 2}, Log: deque([1, 2]), Counter: defaultdict(<class 'int'>, {1: 1, 2: 1})
 - removed item from cache:  2
put(3, 3) 	 | Cache(2): {1: 1, 3: 3}, Log: deque([3, 1]), Counter: defaultdict(<class 'int'>, {1: 1, 3: 1})
get(2) -> -1 	 | Cache(2): {1: 1, 3: 3}, Log: deque([3, 1]), Counter: defaultdict(<class 'int'>, {1: 1, 3: 1})
 - removed item from cache:  1
put(4, 4) 	 | Cache(2): {3: 3, 4: 4}, Log: deque([4, 3]), Counter: defaultdict(<class 'int'>, {3: 1, 4: 1})
get(1) -> -1 	 | Cache(2): {3: 3, 4: 4}, Log: deque([4, 3]), Counter: defaultdict(<class 'int'>, {3: 1, 4: 1})
 - removed dup item from end of access log:  3
get(3) -> 3 	 | Cache(2): {3: 3, 4: 4}, Log: deque([3, 4]), Counter: defaultdict(<class 'int'>,

In [2]:
input_data = [[10],[10,13],[3,17],[6,11],[10,5],[9,10],[13],[2,19],[2],[3],[5,25],[8],[9,22],[5,5],[1,30],[11],[9,12],[7],[5],[8],[9],[4,30],[9,3],[9],[10],[10],[6,14],[3,1],[3],[10,11],[8],[2,14],[1],[5],[4],[11,4],[12,24],[5,18],[13],[7,23],[8],[12],[3,27],[2,12],[5],[2,9],[13,4],[8,18],[1,7],[6],[9,29],[8,21],[5],[6,30],[1,12],[10],[4,15],[7,22],[11,26],[8,17],[9,29],[5],[3,4],[11,30],[12],[4,29],[3],[9],[6],[3,4],[1],[10],[3,29],[10,28],[1,20],[11,13],[3],[3,12],[3,8],[10,9],[3,26],[8],[7],[5],[13,17],[2,27],[11,15],[12],[9,19],[2,15],[3,16],[1],[12,17],[9,1],[6,19],[4],[5],[5],[8,1],[11,7],[5,2],[9,28],[1],[2,2],[7,4],[4,22],[7,24],[9,26],[13,28],[11,26]]
expect_output = [None,None,None,None,None,None,-1,None,19,17,None,-1,None,None,None,-1,None,-1,5,-1,12,None,None,3,5,5,None,None,1,None,-1,None,30,5,30,None,None,None,-1,None,-1,24,None,None,18,None,None,None,None,-1,None,None,18,None,None,-1,None,None,None,None,None,18,None,None,-1,None,4,29,30,None,12,-1,None,None,None,None,29,None,None,None,None,17,22,18,None,None,None,-1,None,None,None,20,None,None,None,-1,18,18,None,None,None,None,20,None,None,None,None,None,None,None]

lru_cache = LRUCache(input_data[0][0])
for input_, expect_ in zip(input_data[1:], expect_output[1:]):
    if len(input_) == 1:
        output_ = lru_cache.get(input_[0])
        if output_ != expect_:
            print('Getting %s -> ' % input_[0], output_, expect_)
    else:
        lru_cache.put(*input_)

put(10, 13) 	 | Cache(1): {10: 13}, Log: deque([10]), Counter: defaultdict(<class 'int'>, {10: 1})
put(3, 17) 	 | Cache(2): {10: 13, 3: 17}, Log: deque([3, 10]), Counter: defaultdict(<class 'int'>, {10: 1, 3: 1})
put(6, 11) 	 | Cache(3): {10: 13, 3: 17, 6: 11}, Log: deque([6, 3, 10]), Counter: defaultdict(<class 'int'>, {10: 1, 3: 1, 6: 1})
 - removed dup item from end of access log:  10
put(10, 5) 	 | Cache(3): {10: 5, 3: 17, 6: 11}, Log: deque([10, 6, 3]), Counter: defaultdict(<class 'int'>, {10: 1, 3: 1, 6: 1})
put(9, 10) 	 | Cache(4): {10: 5, 3: 17, 6: 11, 9: 10}, Log: deque([9, 10, 6, 3]), Counter: defaultdict(<class 'int'>, {10: 1, 3: 1, 6: 1, 9: 1})
get(13) -> -1 	 | Cache(4): {10: 5, 3: 17, 6: 11, 9: 10}, Log: deque([9, 10, 6, 3]), Counter: defaultdict(<class 'int'>, {10: 1, 3: 1, 6: 1, 9: 1})
put(2, 19) 	 | Cache(5): {10: 5, 3: 17, 6: 11, 9: 10, 2: 19}, Log: deque([2, 9, 10, 6, 3]), Counter: defaultdict(<class 'int'>, {10: 1, 3: 1, 6: 1, 9: 1, 2: 1})
get(2) -> 19 	 | Cache(5):